Libraries

In [21]:
import os
import pandas as pd
from IPython.display import display, HTML, Markdown

from langchain_openai import AzureChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

Credentials

In [4]:
model_name = "gpt-4o-mini"
deployment = "gpt-4o-mini"
api_version = "2024-12-01-preview"

# Load credentials from environment variable
subscription_key = os.getenv("subscription_key")
if not subscription_key:
    raise RuntimeError("subscription_key not found in environment variables")

endpoint = os.getenv("endpoint")
if not endpoint:
    raise RuntimeError("endpoint not found in environment variables")

Prep model

In [5]:
llm = AzureChatOpenAI(
    azure_deployment = deployment,
    api_version = api_version,
    api_key = subscription_key,
    azure_endpoint = endpoint,
    # temperature=0,
    # max_tokens=None,
    # timeout=None,
    # max_retries=2,
    # # other params...
)

Test one message and response

In [6]:
messages = [
    ("system", "You are a helpful assistant that translates English to French. Translate the user sentence."),
    ("human", "I love programming.")
]

# call LLM test
ai_msg = llm.invoke(messages)

print("AI Response:", ai_msg)
print(ai_msg.content)

AI Response: content="J'adore programmer." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 31, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7a53abb7a2', 'id': 'chatcmpl-BgGFLcdnQU94bLNr6eyKrtOq2SWSw', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': 

Read initiatives CSV and interact

In [7]:
file_url = "./data/processed/1. initiatives.csv"
df = pd.read_csv(file_url)

print(df.head())

   initiative_id                                              title  \
0         304233             Aprova o Orçamento do Estado para 2025   
1         284017  Procede à interpretação autêntica do n.º 2 do ...   
2         314619  Aprova, para ratificação, as emendas ao Acordo...   
3         304313  Pela responsabilização do Estado na alocação d...   
4         304312  Assegura a execução, na ordem jurídica interna...   

                                             authors                   type  \
0                                            Governo        Proposta de Lei   
1                                            Governo        Proposta de Lei   
2                                            Governo  Proposta de Resolução   
3  Assembleia Legislativa da Região Autónoma da M...        Proposta de Lei   
4                                            Governo        Proposta de Lei   

                                                 url  
0  http://app.parlamento.pt/webutils/docs/d

Prepare the LangChain agent

In [14]:
agent = create_pandas_dataframe_agent(llm=llm, df=df, agent_type="tool-calling", verbose=True, allow_dangerous_code=True)

In [ ]:
agent_response = agent.invoke("How many rows are there?")
print(); print("Agent Response:", agent_response["output"])



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df)'}`


1397There are 1,397 rows in the dataframe.

> Finished chain.
Agent Response: There are 1,397 rows in the dataframe.


Prompt Engineering

In [23]:
PROMPT_INSTRUCTIONS = """
You are an AI assistant with knowledge of legislative initiatives from the Portuguese Parliament.
Each initiative has the following details: initiative_id, title, authors, type, and url.

**Guidelines**:
- Only mention the URL as a source if and only if the user asks about or references a specific initiative (e.g., by title, type, or ID).
- Interpret the field authors with nuance: it may refer to political parties, the Government, parliamentary groups, individual deputies or others.
- The type should be interpreted and presented in Portuguese from Portugal (pt-PT). Translate or explain types if necessary when answering in English.
- Always answer in the same language the user uses (European Portuguese or English).

Your **goal** is to provide accurate, clear, and context-aware information about the initiatives.
"""

PROMPT_REASONING = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself if it answers correctly the original question.
If you are not sure, try another method.

- If the methods tried do not give the same result, reflect and try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful and thorough response using Markdown.

- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get to the final answer.
"""

In [25]:
QUESTION = "Quantas iniciativas foram da responsabilidade do governo?" 

agent_response = agent.invoke(PROMPT_INSTRUCTIONS + PROMPT_REASONING + QUESTION)
print(); print("---"); print("Agent Response"), print()
display(Markdown(agent_response["output"]))



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['authors'] == 'Governo'].shape[0]"}`


35
Invoking: `python_repl_ast` with `{'query': "df['authors'].value_counts().get('Governo', 0)"}`


35**Final Answer:**

Foram registradas 35 iniciativas da responsabilidade do Governo.

**Explanation:**
Para determinar o número de iniciativas do Governo, utilizei a coluna **authors** e contei quantas vezes o termo "Governo" aparece nessa coluna. A função `value_counts()` e um filtro específico foram utilizados, confirmando que o total é de 35 iniciativas.

> Finished chain.

---
Agent Response



**Final Answer:**

Foram registradas 35 iniciativas da responsabilidade do Governo.

**Explanation:**
Para determinar o número de iniciativas do Governo, utilizei a coluna **authors** e contei quantas vezes o termo "Governo" aparece nessa coluna. A função `value_counts()` e um filtro específico foram utilizados, confirmando que o total é de 35 iniciativas.